In [19]:
import theano.tensor as T
import theano
from pymanopt.manifolds import *

s = Stiefel(3, 1)

In [56]:
X = T.matrix()
A = T.matrix()
cost = T.exp(T.sum(X**2))

egrad = T.grad(cost, X)
ehess = T.Rop(egrad, X, A)

XtG = T.dot(X.T, egrad)
rgrad = egrad - T.dot(X, XtG + XtG.T) / 2

drgrad = T.Rop(rgrad, X, A)

rhess = T.Rop(rgrad, X, A)

In [57]:
egrad_func = theano.function([X], egrad)
ehess_func = theano.function([X, A], ehess)
drgrad_func = theano.function([X, A], drgrad)
rhess_func = theano.function([X, A], rhess)

In [63]:
X = s.rand()
A = 20000000*s.randvec(X)

In [64]:
print s.ehess2rhess(X, egrad_func(X), ehess_func(X, A), A)

[[ -2.88666582e-09]
 [  6.32066186e-10]
 [  3.06485066e-09]]


In [65]:
print rhess_func(X, A)

[[  0.00000000e+00]
 [ -7.45058060e-09]
 [  7.45058060e-09]]


In [66]:
def rhess_new(Y, U):
    rhess = s.proj(Y, ehess_func(Y, U))
    G = egrad_func(Y)
    t = -(Y.dot(U.T.dot(G) + G.T.dot(U)) + U.dot(Y.T.dot(G) + G.T.dot(Y)))/2
    return rhess + t
print rhess_new(X, A)

[[  0.00000000e+00]
 [ -7.45058060e-09]
 [  7.45058060e-09]]


In [67]:
print s.proj(X, drgrad_func(X, A))

[[ -2.88666582e-09]
 [  6.32066186e-10]
 [  3.06485066e-09]]
